In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('gurgaon_properties_post_feature_selection.csv')
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,1.0,56.0,4.0,4.0,3.0,3.0,3240.0,1,1,2,2.0,1.0,7.25
1,0.0,45.0,3.0,3.0,2.0,1.0,1593.0,0,0,0,1.0,1.0,0.86
2,0.0,94.0,3.0,3.0,4.0,1.0,1929.0,1,0,1,0.0,2.0,2.25
3,1.0,66.0,9.0,9.0,4.0,1.0,2660.0,1,0,2,2.0,2.0,6.25
4,0.0,90.0,3.0,3.0,2.0,3.0,1730.0,0,0,2,2.0,2.0,0.95


In [3]:
# one hot encode -> sector, balcony, agePossession, furnishing type, luxury category, floor category

In [4]:
X = df.drop(columns=['price'])
y = df['price']

In [5]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [6]:
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [7]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [22]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first',handle_unknown='ignore'), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [24]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

In [26]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [28]:
scores.mean()

0.8804460096767466

In [30]:
scores.std()

0.013506205479335138

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [34]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', SVR())])

In [36]:
y_pred = pipeline.predict(X_test)
y_pred = np.expm1(y_pred)

In [38]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

0.5248779962863319